In [1]:
# обязательно привязываем гугл диск
# для этого необходимо скачать решение к себе!!
from google.colab import drive
drive.mount('/content/drive')
# после строчки кода выше мы можем обращаться к
# файлам через "./drive/My Drive/{ваша папка на диске}"
!pip install catboost # зависимость

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
# ваша/наша папка
folder = "Ситилинк Хакатон/Для организаторов/Ситилинк Хакатон/Рабочее решение"
# "./drive/My Drive/" обязательно должен быть
# по нему мы обращаемся к моделям
path = f"./drive/My Drive/{folder}/"
print(path)

./drive/My Drive/Ситилинк Хакатон/Для организаторов/Ситилинк Хакатон/Рабочее решение/


In [4]:
import os
# список файлов в текущем каталоге
os.listdir(path)

['2016-2021forecast.csv',
 'joined.csv',
 'info.ipynb',
 'joined_last.csv',
 'joined_new.csv',
 'random',
 'torch_train.ipynb',
 'baseline_v1.1.ipynb',
 'Models',
 'ModernClassifier_train.ipynb',
 'ModernRegressor_train.ipynb',
 'baseline_v1_2.ipynb']

In [5]:
import os
from torchvision import transforms
import pickle
import pandas as pd
import torch
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np
import datetime

# использует GPU, с CPU есть баги!
class LakeModel():
    def __init__(self, models_path, train_df_path):
        self.device = torch.device('cuda' if 
                                   torch.cuda.is_available() else 'cpu')
        self.network = torch.load(os.path.join(models_path, "network.pth"))
        self.network = self.network.to(self.device)
        self.network.eval()
        with open(os.path.join(models_path, "ModernClassifier.pkl"), "rb") as f:
            self.classifier = pickle.load(f)
        with open(os.path.join(models_path, "ModernRegressor.pkl"), "rb") as f:
            self.regressor = pickle.load(f)
        self.transform = transforms.ToTensor()
        self.size = (128, 128)
        self.columns = ['hour', 'year', 'week', 'day',
                        'days_in_month', 'month', 'quarter', 'season',
                        'prediction']
        self.df = None
        self.start_date = datetime.datetime(2016, 3, 1)
        df_joined = pd.read_csv(train_df_path)
        df_joined = df_joined[df_joined.y == 1]
        self.dates = df_joined.name\
            .apply(lambda x: datetime.datetime.strptime(x[: -4],
                                                        "%Y%m%d%H%M%S"))

 
    def date_to_data(self, frame, name="date", suffix=""):
        assert name in frame, "Нужна колонка с датой"
        date = frame[name].dt
        info = pd.concat([date.days_in_month,
                        date.month,
                        date.quarter,
                        date.month % 12 // 3 + 1,
                        date.hour], axis=1)
        info.columns = ["days_in_month", "month",
                        "quarter", "season", "hour"]
        temp = pd.concat([date.isocalendar(),
                        info], axis=1)
        temp.columns = temp.columns + suffix
        frame = pd.concat([frame, temp], axis=1)
        return frame
    
    def predict_network(self, path_):
        predictions = []
        for filename in tqdm(os.listdir(path_)):
            with Image.open(os.path.join(path_, filename)) as image:
                image = image.resize(self.size)
            X = self.transform(image)
            X = X.view(1, *X.size()).to(self.device)
            prediction = self.network(X).argmax(dim=1).item()
            predictions.append([filename, prediction])
        return predictions
    
    def check_anomaly(self, output):
        arr = []
        for name, value in output:
            if pd.to_datetime(name[:-4]).month in [6, 7, 8]:
                arr.append([name, 0])
            else:
                arr.append([name, value])
        return arr

    def prepare(self, path_):
        df = pd.DataFrame(os.listdir(path_), columns=["name"])
        df["time_parsed"] = pd.to_datetime(df["name"].apply(lambda x: x[:-4]))
        df = self.date_to_data(df, name="time_parsed")

        predictions = self.predict_network(path_)
        predictions_df = pd.DataFrame(predictions,
                                      columns=["filename", "prediction"])
        df = df.merge(predictions_df, left_on="name", right_on="filename")
  
        self.df = df
    
    def predict_classification(self, path_):
        self.prepare(path_)
        output = zip(self.df["name"], self.classifier.predict(self.df[self.columns]\
                                                         .astype(np.float32)))
        return pd.DataFrame(self.check_anomaly(output),
                            columns=["filename", "prediction"])
    
    def nearest(self, items, pivot):
        return min(items, key=lambda x: abs(x - pivot))

    def time2path(self, d_object):
        obj = self.nearest(self.dates, d_object).replace(minute=0, second=0)
        return datetime.datetime.strftime(obj, "%Y%m%d%H%M") + ".jpg"

    def predict_regreassion(self, path_):
        self.prepare(path_)
        output = zip(self.df["name"], self.regressor.predict(self.df[self.columns]\
                                                             .astype(np.float32)))
        
        output = pd.DataFrame(output, columns=["filename", "prediction"])
        output[output["prediction"] < 0]["prediction"] = 0
        output["prediction"] = np.round(output["prediction"], 2)
        output["time"] = pd.to_datetime(output["filename"].apply(lambda x: x[:-4]))
        output["delta"] = output["prediction"].apply(lambda x: datetime.timedelta(hours=x))
        output["prediction"] = output["time"].dt.to_pydatetime() + output["delta"]
        output["prediction_photo"] = output["prediction"].apply(lambda x: self.time2path(x))
        
        return output[["filename", "prediction", "prediction_photo"]]

In [6]:
#                     путь к моделям    ↓
model = LakeModel(os.path.join(path, "Models"),
#                       размеченный датасет   ↓
                  os.path.join(path, "joined_last.csv"))

# Предсказываем установился лед на озере или нет

In [7]:
# здесь random - папка с фото для тестирования, вы можете ее изменить
predictions = model.predict_network(os.path.join(path, "random"))

In [8]:
predictions

[['201812171600.jpg', 0],
 ['201812171400.jpg', 0],
 ['201812180400.jpg', 1],
 ['201812180600.jpg', 0],
 ['201812171800.jpg', 0],
 ['201812180000.jpg', 1],
 ['201812180200.jpg', 0],
 ['201812172200.jpg', 1],
 ['201812172000.jpg', 1],
 ['201812171200.jpg', 0]]

# Предсказываем дату и время полного замерзания Петрозаводской губы Онежского озера

In [9]:
# здесь random - папка с фото для тестирования, вы можете ее изменить
predictions = model.predict_regreassion(os.path.join(path, "random")) # путь к папке с фото

In [10]:
# prediction_photo показыает название предпоагаемого файла из трейна
# мы не стали его выводить на экран, так как будет не удобно проверять
# на всякий случай мы приложили код для просмотра изображения/изображений в ноутбуке
predictions

,filename,prediction,prediction_photo
0,201812171600.jpg,2018-12-19 05:25:12,201812190600.jpg
1,201812171400.jpg,2018-12-19 01:58:12,201812190200.jpg
2,201812180400.jpg,2018-12-18 14:15:00,201812181400.jpg
3,201812180600.jpg,2018-12-19 06:21:00,201812190600.jpg
4,201812171800.jpg,2018-12-19 05:25:48,201812190600.jpg
5,201812180000.jpg,2018-12-18 15:00:36,201812181600.jpg
6,201812180200.jpg,2018-12-19 10:06:36,201812191000.jpg
7,201812172200.jpg,2018-12-18 19:45:00,201812182000.jpg
8,201812172000.jpg,2018-12-18 19:40:48,201812182000.jpg
9,201812171200.jpg,2018-12-19 09:48:00,201812191000.jpg


In [11]:
# ниже код для скачивания трейна, без него не работает
# from IPython.display import clear_output
# !wget https://disk.karelia.pro/djnSMfz/Трек%201.%20Онежское%20озеро.tgz?attachment
# import os
# try:
#     os.mkdir("./data")
# except: pass
# !tar -xvzf "Трек 1. Онежское озеро.tgz?attachment" -C ./data
# clear_output()

# ниже код для просмотра изображения в ноутбуке
# from IPython.display import display, Image
# display(Image(os.path.join("data", "название файла")))

# ниже код для просмотра изображений в ноутбуке
# from IPython.display import display, Image, clear_output
# from time import sleep
# import os

# for filename in predictions["prediction_photo"]:
#     display(Image(os.path.join("data", filename)))
#     sleep(0.4)
#     clear_output()